In [17]:
import os
import pandas as pd

In [18]:
data_loc_800 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/middle_20"
os.path.isfile(f"{data_loc_800}/data.csv")

True

In [19]:
frames_800 = pd.read_csv(f"{data_loc_800}/data.csv")
frames_800["video"] = [frame_name.split("#")[0] for frame_name in frames_800["frame"]]
frames_800

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
15779,vPorn346#15.jpg,1,vPorn346
15780,vPorn346#16.jpg,1,vPorn346
15781,vPorn346#17.jpg,1,vPorn346
15782,vPorn346#18.jpg,1,vPorn346


In [20]:
agg = {"video": "first", "label": "first", "frame": "size"}
videos_800 = frames_800[["video", "frame", "label"]]
videos_800 = videos_800.groupby("video").aggregate(agg).reset_index(drop=True)
videos_800 = videos_800.rename(columns={"frame": "frame_count"})
videos_800

,video,label,frame_count
0,vNonPorn1,0,20
1,vNonPorn10,0,20
2,vNonPorn100,0,20
3,vNonPorn1006,0,20
4,vNonPorn1007,0,20
...,...,...,...
788,vPorn94,1,20
789,vPorn95,1,20
790,vPorn96,1,20
791,vPorn97,1,20


In [21]:
# Videos from which it was not possible to extract 20 frames
videos_800[videos_800["frame_count"] < 20]

,video,label,frame_count
83,vNonPorn1080,0,12
85,vNonPorn1082,0,12
90,vNonPorn1087,0,11
110,vNonPorn1107,0,11
146,vNonPorn1141,0,12
177,vNonPorn1173,0,12
179,vNonPorn1175,0,11
182,vNonPorn1179,0,11
188,vNonPorn1186,0,12


In [22]:
len(videos_800) # 7 videos with 0 frames extracted

793

In [23]:
len(frames_800[frames_800["label"] == 0]) # Number of non-porn frames

7784

In [24]:
len(frames_800[frames_800["label"] == 1]) # Number of porn frames

8000

In [25]:
data_loc_2k = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/middle_20"
os.path.isfile(f"{data_loc_2k}/data.csv")

True

In [26]:
frames_2k = pd.read_csv(f"{data_loc_2k}/data.csv")
frames_2k["video"] = [frame_name.split("#")[0] for frame_name in frames_2k["frame"]]
frames_2k

,frame,label,video
0,vPorn000842#0.jpg,1,vPorn000842
1,vPorn000842#1.jpg,1,vPorn000842
2,vPorn000842#2.jpg,1,vPorn000842
3,vPorn000842#3.jpg,1,vPorn000842
4,vPorn000842#4.jpg,1,vPorn000842
...,...,...,...
39995,vPorn000431#15.jpg,1,vPorn000431
39996,vPorn000431#16.jpg,1,vPorn000431
39997,vPorn000431#17.jpg,1,vPorn000431
39998,vPorn000431#18.jpg,1,vPorn000431


In [27]:
agg = {"video": "first", "label": "first", "frame": "size"}
videos_2k = frames_2k[["video", "frame", "label"]]
videos_2k = videos_2k.groupby("video").aggregate(agg).reset_index(drop=True)
videos_2k = videos_2k.rename(columns={"frame": "frame_count"})
videos_2k

,video,label,frame_count
0,vNonPorn000001,0,20
1,vNonPorn000002,0,20
2,vNonPorn000003,0,20
3,vNonPorn000004,0,20
4,vNonPorn000005,0,20
...,...,...,...
1995,vPorn000996,1,20
1996,vPorn000997,1,20
1997,vPorn000998,1,20
1998,vPorn000999,1,20


In [28]:
# Videos from which it was not possible to extract 40 frames
videos_2k[videos_2k["frame_count"] < 40]

,video,label,frame_count
0,vNonPorn000001,0,20
1,vNonPorn000002,0,20
2,vNonPorn000003,0,20
3,vNonPorn000004,0,20
4,vNonPorn000005,0,20
...,...,...,...
1995,vPorn000996,1,20
1996,vPorn000997,1,20
1997,vPorn000998,1,20
1998,vPorn000999,1,20


In [29]:
# Videos from which it was extracted more than 40 frames
videos_2k[videos_2k["frame_count"] > 40]

,video,label,frame_count


In [31]:
len(frames_2k[frames_2k["label"] == 0]) # Number of non-porn frames

20000

In [32]:
len(frames_2k[frames_2k["label"] == 1]) # Number of porn frames

20000